# 【問題1】SimpleRNNのフォワードプロパゲーション実装

SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。


フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。


バッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。

$$
a_t = x_{t}\cdot W_{x} + h_{t-1}\cdot W_{h} + B\\
h_t = tanh(a_t)
$$

𝑎𝑡 : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)


ℎ𝑡 : 時刻tの状態・出力 (batch_size, n_nodes)


𝑥𝑡 : 時刻tの入力 (batch_size, n_features)


𝑊𝑥 : 入力に対する重み (n_features, n_nodes)


ℎ𝑡−1 : 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)


𝑊ℎ : 状態に対する重み。 (n_nodes, n_nodes)


𝐵 : バイアス項 (n_nodes,)


初期状態 ℎ0 は全て0とすることが多いですが、任意の値を与えることも可能です。


上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力 𝑥 は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。


分類問題であれば、それぞれの時刻のhに対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。タスクによっては最後の時刻のhだけを使用することもあります。



In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from copy import deepcopy

In [2]:
class SimpleRNN:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    Wx : 次の形のndarray, shape (n_features, n_nodes)
    Wh : 次の形のndarray, shape (n_nodes, n_nodes)

    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, Wx, Wh, B, optimizer=None, layer=None):
        self.optimizer = optimizer
        self.layer = layer 

        self.B = B
        self.Wx = Wx
        self.Wh = Wh
        
    def forward(self, X, h_s, A_list=[], cnt = 0):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_sequences, n_features)
            入力
        h_prev : 次の形のndarray, shape (batch_size, n_nodes)
                 入力

        Returns
        ----------
        h : 次の形のndarray, shape (batch_size, n_nodes)
            出力
        """
        self.X = X

        batch_size = X.shape[0]
        n_sequences = X.shape[1]
        n_features = X.shape[2]
        n_nodes = self.Wx.shape[1]

        A = X[:, cnt, :] @ self.Wx + h_s[:, cnt-1, :] @ self.Wh + self.B
        A_list.append(A.tolist())
        h_s[:, cnt, :] = np.tanh(A) #ハイパボリックタンジェント関数

        self.A = np.array(A_list)

        cnt += 1
        if cnt == n_sequences:
            h = h_s[:, cnt-1, :]
            return h
        
        return self.forward(X, h_s, A_list, cnt = cnt)

# 【問題2】小さな配列でのフォワードプロパゲーションの実験
小さな配列でフォワードプロパゲーションを考えてみます。


入力x、初期状態h、重みw_xとw_h、バイアスbを次のようにします。


ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。

In [3]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
h_s = np.zeros((batch_size, n_sequences, n_nodes))
b = np.array([1, 1, 1, 1]) # (n_nodes,)

フォワードプロパゲーションの出力が次のようになることを作成したコードで確認してください。

In [4]:
h = np.array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]]) # (batch_size, n_nodes)

In [5]:
#フォワードプロパゲーション
SRNN = SimpleRNN(w_x, w_h, b)
h = SRNN.forward(x, h_s)
print(h)

[[0.79494228 0.81839002 0.83939649 0.85584174]]


### 出力結果が同じになることを確認した。

# 【問題3】（アドバンス課題）バックプロパゲーションの実装
バックプロパゲーションを実装してください。


RNNの内部は全結合層を組み合わせた形になっているので、更新式は全結合層などと同様です。

$$
W_x^{\prime} = W_x - \alpha \frac{\partial L}{\partial W_x} \\
W_h^{\prime} = W_h - \alpha \frac{\partial L}{\partial W_h} \\
B^{\prime} = B - \alpha \frac{\partial L}{\partial B}
$$

𝛼 : 学習率


$\frac{\partial L}{\partial W_x}$ : 𝑊𝑥 に関する損失 𝐿 の勾配


$\frac{\partial L}{\partial W_h}$ : 𝑊ℎ に関する損失 𝐿 の勾配


$\frac{\partial L}{\partial B}$ : 𝐵 に関する損失 𝐿 の勾配


勾配を求めるためのバックプロパゲーションの数式が以下です。

$
\frac{\partial h_t}{\partial a_t} = \frac{\partial L}{\partial h_t} ×(1-tanh^2(a_t))
$

$
\frac{\partial L}{\partial B} = \frac{\partial h_t}{\partial a_t}
$

$
\frac{\partial L}{\partial W_x} = x_{t}^{T}\cdot \frac{\partial h_t}{\partial a_t}
$

$
\frac{\partial L}{\partial W_h} = h_{t-1}^{T}\cdot \frac{\partial h_t}{\partial a_t}
$

＊$\frac{\partial L}{\partial h_t}$ は前の時刻からの状態の誤差と出力の誤差の合計です。hは順伝播時に出力と次の層に伝わる状態双方に使われているからです。

前の時刻や層に流す誤差の数式は以下です。

$
\frac{\partial L}{\partial h_{t-1}} = \frac{\partial h_t}{\partial a_t}\cdot W_{h}^{T}
$

$
\frac{\partial L}{\partial x_{t}} = \frac{\partial h_t}{\partial a_t}\cdot W_{x}^{T}
$

### 魚本２を参考に作成（写経）

In [13]:
class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.dot(h_prev, Wh) + np.dot(x, Wx) + b
        h_next = np.tanh(t)

        self.cache = (x, h_prev, h_next)
        return h_next

    def backward(self, dh_next):
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache

        dt = dh_next * (1 - h_next ** 2)
        db = np.sum(dt, axis=0)
        dWh = np.dot(h_prev.T, dt)
        dh_prev = np.dot(dt, Wh.T)
        dWx = np.dot(x.T, dt)
        dx = np.dot(dt, Wx.T)

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        return dx, dh_prev


class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None

        self.h, self.dh = None, None
        self.stateful = stateful

    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape
        D, H = Wx.shape

        self.layers = []
        hs = np.empty((N, T, H), dtype='f')

        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f')

        for t in range(T):
            layer = RNN(*self.params)
            self.h = layer.forward(xs[:, t, :], self.h)
            hs[:, t, :] = self.h
            self.layers.append(layer)

        return hs

    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D, H = Wx.shape

        dxs = np.empty((N, T, D), dtype='f')
        dh = 0
        grads = [0, 0, 0]
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:, t, :] + dh)
            dxs[:, t, :] = dx

            for i, grad in enumerate(layer.grads):
                grads[i] += grad

        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        self.dh = dh

        return dxs

    def set_state(self, h):
        self.h = h

    def reset_state(self):
        self.h = None

In [30]:
TRNN = TimeRNN(w_x, w_h, b, stateful=False)
hs = TRNN.forward(x)
hs

array([[[0.76188797, 0.76213956, 0.762391  , 0.7625584 ],
        [0.792209  , 0.8141834 , 0.8340491 , 0.84977716],
        [0.79494226, 0.81839   , 0.8393965 , 0.85584176]]], dtype=float32)

In [32]:
y = np.array([1, 1, 1, 1])
dhs = y - hs
dxs = TRNN.backward(dhs)
dxs

array([[[0.01684319, 0.02416234],
        [0.00868361, 0.01314551],
        [0.00762592, 0.01166947]]], dtype=float32)